In [33]:
# read binary data
import numpy as np

def read_file(filename):
    with open(filename, 'rb') as file:
        # Read the header
        header = file.read(0x1ca)
        # Read the data
        data = file.read()
        # Convert the data to a numpy array
        data = np.frombuffer(data, dtype=np.int16)
        return data

In [56]:
import matplotlib.pyplot as plt
# Define a simple moving average function
def moving_average(data, window_size=3):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


data = read_file('TEK0000RF1.isf')
print(data.shape)
data = data[530000:1790000].copy()
data[data > 75] = 0
# Smooth the data
data = moving_average(data, window_size=3)
# convert to byte array
data = data.astype(np.uint8)
# Make binary
data = np.where(data < 20, 0, 1)
#save data
#np.savetxt("data.csv", adata, delimiter=",", header="Value,Elapsed_Time_us", comments='', fmt='%i,%.2f')
#np.savetxt("data.csv", adata, delimiter=",", header="Value", comments='', fmt='%i')



(5000000,)


In [57]:
compressed_data = []
durations = []

# Iterate through the data and compress
i = 0
while i < len(data):
    start = i
    # Move `i` while the next data point is the same as the current one
    while i < len(data)-1 and data[i] == data[i+1]:
        i += 1
    # Include the last data point if it's the same as the one before
    end = i
    i += 1
    
    compressed_data.append(data[start])
    # Compute the duration of the sequence and store it
    duration = (end - start + 1) * 0.8
    durations.append(duration)

# Pair each compressed data point with its duration
paired_data = np.column_stack((compressed_data, durations))


# Save the data to a CSV file
np.savetxt('data.csv', paired_data, delimiter=',', header='Signal,Elapsed Time', fmt='%i,%.2f')